In [1]:
from spektral import datasets
from spektral.layers import CensNetConv

# Load the citation data.
dataset = datasets.citation.Citation("cora", normalize_x=True, random_splits=True)
cora = dataset.read()[0]

/home/daniel/.pyenv/versions/3.8.6/lib/python3.8/pickle.py:1581: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  return getattr(sys.modules[module], name)
/home/daniel/git/cotton_flower_mot/.venv/lib/python3.8/site-packages/spektral/utils/io.py:25: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  return pickle.load(f, encoding="latin1")
/home/daniel/git/cotton_flower_mot/.venv/lib/python3.8/site-packages/spektral/datasets/citation.py:108: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  a = nx.adjacency_matrix(nx.from_dict_of_lists(graph))  # CSR
/home/daniel/git/cotton_flower_mot/.venv/lib/python3.8/site-packages/networkx/linalg/graphmatrix.py:173: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `to_scipy_sparse

Pre-processing node features
Pre-processing node features


/home/daniel/git/cotton_flower_mot/.venv/lib/python3.8/site-packages/spektral/datasets/citation.py:189: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.array(mask, dtype=np.bool)


In [2]:
import tensorflow as tf

# Convert citation graph into an undirected graph.
adjacency = cora.a.todense()
adjacency_upper = tf.linalg.band_part(adjacency, 0, -1)
adjacency_lower = tf.linalg.band_part(adjacency, -1, 0)

adjacency_upper_symmetric = adjacency_upper + tf.transpose(adjacency_upper)
adjacency_lower_symmetric = adjacency_lower + tf.transpose(adjacency_lower)
adjacency_undirected = tf.maximum(adjacency_upper_symmetric, adjacency_lower_symmetric)

2022-06-21 10:37:25.031261: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 10:37:25.032900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-21 10:37:25.033191: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-21 10:37:25.033425: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-21 10:37:25.033655: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [3]:
from cotton_flower_mot.pipelines.model_training.graph_utils import compute_pairwise_similarities
from cotton_flower_mot.pipelines.model_training.similarity_utils import cosine_similarity
from spektral.data import Graph
tf.keras.mixed_precision.set_global_policy("float32")

# Compute edge features.
connected_node_indices = tf.where(adjacency_upper)
# Get the corresponding node features for each edge.
left_node_features = tf.gather(cora.x, connected_node_indices[:, 0])
right_node_features = tf.gather(cora.x, connected_node_indices[:, 1])
# Compute cosine similarities for each edge.
cosine_similarities = cosine_similarity(left_node_features, right_node_features)

edge_features = tf.expand_dims(cosine_similarities, -1)
cora = Graph(x=cora.x, a=cora.a, e=edge_features.numpy(), y=cora.y)

2022-06-21 10:37:33,357 - tensorflow - WARNING - Mixed precision compatibility check (mixed_float16): WARNING
The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [4]:
from tensorflow.keras import Input, Model, layers
from spektral.layers import CensNetConv

# Build the model.
node_features = Input(shape=(cora.n_node_features,))
edge_features = Input(shape=(cora.n_edge_features,))
node_laplacian = Input(shape=(cora.n_nodes,))
# The undirected graph means that the number of edges is doubled.
edge_laplacian = Input(shape=(cora.n_edges // 2,))
incidence = Input(shape=(cora.n_edges // 2,))

static_features = (node_laplacian, edge_laplacian, incidence)

nodes_2, edges_2 = CensNetConv(64, 64, activation="relu")((node_features, static_features, edge_features))
nodes_2 = layers.Dropout(0.5)(nodes_2)
edges_2 = layers.Dropout(0.5)(edges_2)
nodes_3, _ = CensNetConv(64, 64, activation="relu")((nodes_2, static_features, edges_2))
nodes_3 = layers.Dropout(0.5)(nodes_3)
# Apply the classification.
node_class = layers.Dense(cora.n_labels, activation="softmax")(nodes_3)

model = Model(inputs=[node_features, edge_features, node_laplacian, edge_laplacian, incidence], outputs=[node_class])

2022-06-21 10:37:40,905 - tensorflow - WARNING - AutoGraph could not transform <bound method CensNetConv.call of <spektral.layers.convolutional.censnet_conv.CensNetConv object at 0x7f675c5b1730>> and will run it as-is.
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [23]:
from spektral.data.loaders import SingleLoader
import numpy as np

# Prepare for training.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(reduction="sum"),
              weighted_metrics=["acc"])

# We convert the binary masks to sample weights so that we can compute the
# average loss over the nodes (following original implementation by
# Kipf & Welling)
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)


weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

node_laplacian, edge_laplacian, incidence = CensNetConv.preprocess(cora.a.todense())

inputs_and_targets = ((cora.x, cora.e, node_laplacian, edge_laplacian, incidence), cora.y)
training_dataset = tf.data.Dataset.from_tensors(inputs_and_targets + (weights_tr,))
testing_dataset = tf.data.Dataset.from_tensors(inputs_and_targets + (weights_te,))

In [24]:
tf.keras.mixed_precision.set_global_policy("float32")

# Train the model.
model.fit(training_dataset, validation_data=testing_dataset, epochs=70)

Epoch 1/70
2022-06-20 14:16:02,296 - tensorflow - WARNING - Gradients do not exist for variables ['cens_net_conv_7/edge_kernel:0', 'cens_net_conv_7/node_weights:0', 'cens_net_conv_7/node_bias:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss`argument?
2022-06-20 14:16:02,453 - tensorflow - WARNING - Gradients do not exist for variables ['cens_net_conv_7/edge_kernel:0', 'cens_net_conv_7/node_weights:0', 'cens_net_conv_7/node_bias:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss`argument?
1/1 [==============================] - 3s 3s/step - loss: 1.9459 - acc: 0.1429 - val_loss: 1.9414 - val_acc: 0.3190
Epoch 2/70
1/1 [==============================] - 2s 2s/step - loss: 1.9462 - acc: 0.1500 - val_loss: 1.9435 - val_acc: 0.3190
Epoch 3/70
1/1 [==============================] - 2s 2s/step - loss: 1.9452 - acc: 0.1286 - val_loss: 1.9460 - val_acc: 0.0910
Epoch 4/70
1/1 [=======================